# PyTorch Workflow

Let's explore a an example PyTorch end-to-end workflow.

Resources:
* Ground truth notebook - https://github.com
* Book version of notebook - https://www.learnpytorch.io
* Ask a question - https://github.com

In [ ]:
what_were_covering = {1: "data (prepare and load)",
                      2: "build model",
                      3: "fitting the model to data (training)",
                      4: "making predictions and evaluting a model (infrerence)",
                      5: "saving and loading a model",
                      6: "puttig it all together"}
what_were_covering

{1: 'data (prepare and load)',
 2: 'build model',
 3: 'fitting the model to data (training)',
 4: 'making predictions and evaluting a model (infrerence)',
 5: 'saving and loading a model',
 6: 'puttig it all together'}

In [1]:
import torch
from torch import nn # nn contins all of PyTorch's building blocks for neural networks 
import matplotlib.pyplot as plt

# Check PyTorch version
torch.__version__

'2.0.0+cu118'

## 1. Data (preparing and loading)